In [3]:
import os
import time
from os.path import join

import numpy as np
import torch
from sklearn.model_selection import train_test_split
from tensorboardX import SummaryWriter

import attack as wfpattack
from argparser import parse_taskname, trainparser
from dataset import TraceDataset
import joblib

# prase arguments
# args = trainparser().parse_args()
for taskname in ["meanfix10", "meanfix20", "meanfix40", "meanfix80"]:
    random_seed = 11
    log_root = "./run"
    ds_root = "./data"
    dump_root = "./data/dump"
    cache_root = "./data/cache"
    time_str = time.strftime("%Y%m%d", time.localtime())
    taskname = taskname
    log_dir = join(log_root, "moving_average", "RF", taskname)
    dump_dir = join(dump_root, "moving_average", "RF", taskname)

    attack: wfpattack.Attack = wfpattack.get_attack("RF")(0, 100, 7)

    datas = [joblib.load(f"data/design/{taskname}.pkl")]
    ds_len = len(datas[0]["labels"])
    train_slice, valid_slice = train_test_split(
        [i for i in range(ds_len)], test_size=0.2, random_state=random_seed
    )
    train_features = attack.concat(
        [data["traces"][train_slice][i::2] for i, data in enumerate(datas)]
    )
    train_labels = attack.concat(
        [data["labels"][train_slice][i::2] for i, data in enumerate(datas)]
    )
    valid_features = attack.concat(
        [data["traces"][valid_slice][i::2] for i, data in enumerate(datas)]
    )
    valid_labels = attack.concat(
        [data["labels"][valid_slice][i::2] for i, data in enumerate(datas)]
    )

    attack.init_model()

    writer = SummaryWriter(log_dir)
    attack.train(
        train_features,
        train_labels,
        valid_features,
        valid_labels,
        writer=writer,
        save_root=dump_dir,
    )

training:  81%|████████  | 243/300 [09:15<02:10,  2.29s/it]
